# Word2Vec in Contextual Chatbot

Implementation of word2vec in contextual chatbot.

## Imports

In [1]:
# for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# for tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

# for files
import json

# model export / import
import pickle

## Download NLTK Data

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Import Intents

In [3]:
with open('data/intents.json') as json_data:
    intents = json.load(json_data)

## Organize documents, words and Clasif classes

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']

# go through intents
for intent in intents['intents']:
    # go through each sentence in pattern
    for pattern in intent['patterns']:
        # like: greeting, goodbye, thanks
        class_of_intent = intent['tag']
        
        # tokenize words in the sentence
        words_in_pattern = nltk.word_tokenize(pattern)
        
        # add to global words list
        words.extend(words_in_pattern)
        
        # add to documents in our corpus
        documents.append( (words_in_pattern, class_of_intent) )
        
        # add to classes list
        if class_of_intent not in classes:
            classes.append(class_of_intent)
      
    
# stem, lower and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print(len(documents), " documents")
print("---")
print(len(classes), "classes", classes)
print("---")
print(len(words), "unique stemmed words", words)
        

27  documents
---
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
---
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


## Transform Data to Tensorflow

In [5]:
# training data
training = []
output = []

# tensor of classes
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # stem words
    stem_pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    # create bag of words array
    for w in words:
        bag.append(1) if w in stem_pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


## Build Model

In [6]:
# reset underlying graph data
tf.reset_default_graph()

# Build Neural Network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start Training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('exports/model.tflearn')

Training Step: 3999  | total loss: 0.59890 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.59890 - acc: 0.9666 -- iter: 24/27
Training Step: 4000  | total loss: 0.54157 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.54157 - acc: 0.9699 -- iter: 27/27
--
INFO:tensorflow:/notebooks/NLP/ContextualChatbot/exports/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Save Data-structures

- words
- classess
- training data

In [7]:
pickle.dump( { 'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y }, open( "data/training_data.pkl", "wb" ) )